In [36]:
import pandas as pd
import numpy as np

## Carga de Datos

In [37]:
# redefinir los tipos de datos para las columnas que se cargan como 'object'
dtype_spec_test = {
    'column_6_name': 'str',  # Replace 'column_6_name' with the actual name of column 6 in merged_test.csv
}

dtype_spec_train = {
    'column_7_name': 'str',  # Replace 'column_7_name' with the actual name of column 7 in merged_train.csv
}

# cargar los datos
clean_test = pd.read_csv('merged_test.csv', dtype=dtype_spec_test, low_memory=False)
clean_train = pd.read_csv('merged_train.csv', dtype=dtype_spec_train, low_memory=False)


## Limpieza de Datos


In [38]:
#pasar nombre de columnas a minusculas y reemplazar espacios por guiones bajos (preferencia de la autora) 
clean_test.columns = clean_test.columns.str.lower().str.replace(' ', '_')
clean_train.columns = clean_train.columns.str.lower().str.replace(' ', '_')

In [39]:
#evaluamos la presencia de valores nulos
print(clean_test.isnull().sum())
print(clean_train.isnull().sum())

id                               0
store                            0
dayofweek                        0
date                             0
open                            11
promo                            0
stateholiday                     0
schoolholiday                    0
storetype                        0
assortment                       0
competitiondistance             96
competitionopensincemonth    15216
competitionopensinceyear     15216
promo2                           0
promo2sinceweek              17232
promo2sinceyear              17232
promointerval                17232
dtype: int64
store                             0
dayofweek                         0
date                              0
sales                             0
customers                         0
open                              0
promo                             0
stateholiday                      0
schoolholiday                     0
storetype                         0
assortment                      

In [40]:
#pasar la columna date a datetime
clean_test['date'] = pd.to_datetime(clean_test['date'])
clean_train['date'] = pd.to_datetime(clean_train['date'])

In [41]:
#reemplazar los valores nulos de la columna competitiondistance por la mediana
clean_test['competitiondistance'].fillna(clean_test['competitiondistance'].median(), inplace=True)
clean_train['competitiondistance'].fillna(clean_train['competitiondistance'].median(), inplace=True)

/var/folders/dk/_8pjx3z94934mw25lyqb9xkw0000gn/T/ipykernel_2401/449920827.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  clean_test['competitiondistance'].fillna(clean_test['competitiondistance'].median(), inplace=True)
/var/folders/dk/_8pjx3z94934mw25lyqb9xkw0000gn/T/ipykernel_2401/449920827.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermedi

In [42]:
clean_train.head()

,store,dayofweek,date,sales,customers,open,promo,stateholiday,schoolholiday,storetype,assortment,competitiondistance,competitionopensincemonth,competitionopensinceyear,promo2,promo2sinceweek,promo2sinceyear,promointerval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,5,2015-07-31,4822,559,1,1,0,1,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


## Preprocesamiento de datos

In [43]:
# convertir 'date' a datetime
clean_train['date'] = pd.to_datetime(clean_train['date'])
clean_test['date'] = pd.to_datetime(clean_test['date'])

# feature eng a partir de la fecha
for df in [clean_train, clean_test]:
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['day_of_week'] = df['date'].dt.dayofweek
    df['day_of_year'] = df['date'].dt.dayofyear
    df['quarter'] = df['date'].dt.quarter
    df['is_month_start'] = df['date'].dt.is_month_start
    df['is_month_end'] = df['date'].dt.is_month_end
    df['is_quarter_start'] = df['date'].dt.is_quarter_start
    df['is_quarter_end'] = df['date'].dt.is_quarter_end
    df['is_year_start'] = df['date'].dt.is_year_start
    df['is_year_end'] = df['date'].dt.is_year_end


In [44]:
# asegurarme que 'date' este en datetime (me tiraba error en los modelos sino)
clean_train['date'] = pd.to_datetime(clean_train['date'])
clean_test['date'] = pd.to_datetime(clean_test['date'])

# mas feature eng a partir de las fechas
clean_train['month'] = clean_train['date'].dt.month
clean_train['day_of_week'] = clean_train['date'].dt.dayofweek

clean_test['month'] = clean_test['date'].dt.month
clean_test['day_of_week'] = clean_test['date'].dt.dayofweek

In [45]:
# creacion de features a partir de holiday events
for df in [clean_train, clean_test]:
    df['is_christmas'] = df['month'] == 12

# promedio de ventas por store
store_avg_sales = clean_train.groupby('store')['sales'].mean().reset_index()
store_avg_sales.columns = ['store', 'avg_sales_per_store']
clean_train = clean_train.merge(store_avg_sales, on='store', how='left')
clean_test = clean_test.merge(store_avg_sales, on='store', how='left')


In [46]:

# creacion de feature de 'storetype' y 'promo' 
for df in [clean_train, clean_test]:
    df['storetype_promo'] = df.apply(
        lambda row: f"storetype_{row['storetype']}_haspromo" if row['promo'] == 1 else f"storetype_{row['storetype']}_nopromo",
        axis=1
    )
    df = pd.get_dummies(df, columns=['storetype_promo'], drop_first=False)


In [47]:
# one-hot encoding de 'storetype', 'assortment', y 'promointerval' 
categorical_columns = ['storetype', 'assortment', 'promointerval']
for df in [clean_train, clean_test]:
    df = pd.get_dummies(df, columns=categorical_columns, drop_first=True)

# convertir 'promo2sinceweek' y 'promo2sinceyear' a features de datetime 
for df in [clean_train, clean_test]:
    df['promo2_start_date'] = df.apply(
        lambda row: pd.to_datetime(f"{int(row['promo2sinceyear'])}{int(row['promo2sinceweek']):02}1", format='%Y%W%w', errors='coerce')
        if pd.notna(row['promo2sinceweek']) and pd.notna(row['promo2sinceyear'])
        else pd.NaT,
        axis=1
    )
    df['promo2_start_year'] = df['promo2_start_date'].dt.year
    df['promo2_start_month'] = df['promo2_start_date'].dt.month
    df['promo2_start_day'] = df['promo2_start_date'].dt.day
    df['promo2_start_dayofweek'] = df['promo2_start_date'].dt.dayofweek


In [48]:
# features adicionales a partir de la 'date0
for df in [clean_train, clean_test]:
    df['quarter'] = df['date'].dt.quarter
    df['week_of_year'] = df['date'].dt.isocalendar().week
    df['is_weekend'] = df['day_of_week'] >= 5

# features a partir de Promo2 
month_names = {
    1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr',
    5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug',
    9: 'Sept', 10: 'Oct', 11: 'Nov', 12: 'Dec'
}
for df in [clean_train, clean_test]:
    df['month_name'] = df['month'].map(month_names)
    df['is_promo2_active'] = df.apply(
        lambda row: 1 if (row['promo2'] == 1 and pd.notna(row['promointerval']) and row['month_name'] in row['promointerval']) else 0,
        axis=1
    )

In [49]:
# desde cuando la competencia esta abierta
for df in [clean_train, clean_test]:
    df['competition_open_since'] = ((df['year'] - df['competitionopensinceyear']) * 12) + (df['month'] - df['competitionopensincemonth'])
    df['competition_open_since'] = df['competition_open_since'].apply(lambda x: x if x > 0 else 0)
    df['competition_open_since'].fillna(0, inplace=True)

# encoding de Assortment Level 
assortment_mapping = {'a': 1, 'b': 2, 'c': 3}
for df in [clean_train, clean_test]:
    df['assortment_encoded'] = df['assortment'].map(assortment_mapping)


/var/folders/dk/_8pjx3z94934mw25lyqb9xkw0000gn/T/ipykernel_2401/1973295688.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['competition_open_since'].fillna(0, inplace=True)
/var/folders/dk/_8pjx3z94934mw25lyqb9xkw0000gn/T/ipykernel_2401/1973295688.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values al

In [50]:
# binning a partir de Competition Distance 
distance_bins = [0, 2000, 5000, 10000, np.inf]
distance_labels = ['very_close', 'close', 'far', 'very_far']
for df in [clean_train, clean_test]:
    df['competition_distance_bin'] = pd.cut(df['competitiondistance'], bins=distance_bins, labels=distance_labels)
    df = pd.get_dummies(df, columns=['competition_distance_bin'], drop_first=True)

# matching de Promo Interval
for df in [clean_train, clean_test]:
    df['is_in_promo_interval'] = df.apply(
        lambda row: 1 if pd.notna(row['promointerval']) and row['month_name'] in row['promointerval'].split(',') else 0,
        axis=1
    )

In [51]:
# cyclical encoding para 'day_of_week' -> leimos en la discusion que funcionaba bien
for df in [clean_train, clean_test]:
    df['day_of_week_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
    df['day_of_week_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)


In [52]:
# interaction terms entre storetype y assortment
for df in [clean_train, clean_test]:
    df['storetype_assortment'] = df['storetype'] + '_' + df['assortment']
    df = pd.get_dummies(df, columns=['storetype_assortment'], drop_first=True)

In [53]:
# guardar el dataset
clean_train.to_csv('train_eng.csv', index=False)
clean_test.to_csv('test_eng.csv', index=False)